In [ ]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Read the training and test data
train_csv_file = './train_means_list.csv'
train_df = pd.read_csv(train_csv_file)

train_df['image'] = train_df['image'].str.lstrip('/')
print(train_df.head())
test_csv_file = './test_list.csv'
test_df = pd.read_csv(test_csv_file)
test_df['image'] = test_df['image'].str.lstrip('/')

# Set the data directory
data_dir = 'resized'
num_classes = 9  # Number of output classes
# Convert mean_score to categorical labels
labels = train_df['mean_score']
mean_score_labels = pd.qcut(labels, q=num_classes, labels=False, duplicates='drop')
train_df['label'] = mean_score_labels.astype(str)

# Define the network architecture
def create_model(num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Define training parameters

input_size = (224, 224)  # Input image size
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Prepare the dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # other data augmentation and preprocessing options if needed
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=input_size,
    batch_size=batch_size,
    class_mode="categorical",
)

# Perform k-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds)

for fold, (train_index, val_index) in enumerate(skf.split(train_df, train_df['label'])):
    print(f"Fold {fold + 1}/{num_folds}")

    # Create data generators for the current fold
    train_generator_fold = train_datagen.flow_from_dataframe(
        dataframe=train_df.iloc[train_index],
        directory=data_dir,
        x_col="image",
        y_col="label",
        target_size=input_size,
        batch_size=batch_size,
        class_mode="categorical",
    )
    val_generator_fold = train_datagen.flow_from_dataframe(
        dataframe=train_df.iloc[val_index],
        directory=data_dir,
        x_col="image",
        y_col="label",
        target_size=input_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False,
    )

    # Initialize the model
    model = create_model(num_classes)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

    # Training loop
    model.fit(train_generator_fold,
              epochs=num_epochs,
              validation_data=val_generator_fold,
              verbose=1)

    # Save the model after each fold
    fold_model_path = f"model_fold{fold + 1}.h5"
    model.save(fold_model_path)
    print(f"Model saved at {fold_model_path}")


                     image  mean_score
0  english_resized/143.png    4.483634
1   english_resized/75.png    2.308903
2   english_resized/24.png    3.789444
3  english_resized/196.png    4.784378
4  english_resized/282.png    5.743757
Found 300 validated image filenames belonging to 9 classes.
Fold 1/5
Found 240 validated image filenames belonging to 9 classes.
Found 60 validated image filenames belonging to 9 classes.
Epoch 1/10


2023-06-15 20:46:10.619146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - ETA: 0s - loss: 4.5550 - accuracy: 0.1083

2023-06-15 20:46:53.509901: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 44s 5s/step - loss: 4.5550 - accuracy: 0.1083 - val_loss: 2.2007 - val_accuracy: 0.1167
Epoch 2/10
8/8 [==============================] - 41s 5s/step - loss: 2.6957 - accuracy: 0.1042 - val_loss: 2.3783 - val_accuracy: 0.0833
Epoch 3/10
8/8 [==============================] - 41s 5s/step - loss: 2.1977 - accuracy: 0.1125 - val_loss: 2.1976 - val_accuracy: 0.1167
Epoch 4/10
8/8 [==============================] - 42s 5s/step - loss: 2.1995 - accuracy: 0.1125 - val_loss: 2.1980 - val_accuracy: 0.1167
Epoch 5/10
8/8 [==============================] - 41s 5s/step - loss: 2.1980 - accuracy: 0.1125 - val_loss: 2.1979 - val_accuracy: 0.1167
Epoch 6/10
8/8 [==============================] - 41s 5s/step - loss: 2.1981 - accuracy: 0.1167 - val_loss: 2.1978 - val_accuracy: 0.1167
Epoch 7/10
8/8 [==============================] - 41s 5s/step - loss: 2.1973 - accuracy: 0.1167 - val_loss: 2.1977 - val_accuracy: 0.1167
Epoch 8/10
8/8 [=============================

...

8/8 [==============================] - 43s 5s/step - loss: 5.5621 - accuracy: 0.0958 - val_loss: 2.1970 - val_accuracy: 0.1167
Epoch 2/10
8/8 [==============================] - 41s 5s/step - loss: 2.9210 - accuracy: 0.1083 - val_loss: 2.1945 - val_accuracy: 0.1167
Epoch 3/10
8/8 [==============================] - 41s 5s/step - loss: 2.2000 - accuracy: 0.1167 - val_loss: 2.1976 - val_accuracy: 0.1167
Epoch 4/10
8/8 [==============================] - 41s 5s/step - loss: 2.2353 - accuracy: 0.1125 - val_loss: 2.1985 - val_accuracy: 0.1167
Epoch 5/10
8/8 [==============================] - 41s 5s/step - loss: 2.1978 - accuracy: 0.1083 - val_loss: 2.1990 - val_accuracy: 0.1167
Epoch 6/10
8/8 [==============================] - 41s 5s/step - loss: 2.2125 - accuracy: 0.1000 - val_loss: 2.1989 - val_accuracy: 0.1000
Epoch 7/10
8/8 [==============================] - 41s 5s/step - loss: 2.2025 - accuracy: 0.1042 - val_loss: 2.1988 - val_accuracy: 0.1000
Epoch 8/10
8/8 [=============================

2023-06-15 21:00:00.036950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - ETA: 0s - loss: 6.1883 - accuracy: 0.1000

2023-06-15 21:00:41.909502: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 43s 5s/step - loss: 6.1883 - accuracy: 0.1000 - val_loss: 2.3616 - val_accuracy: 0.1167
Epoch 2/10
8/8 [==============================] - 40s 5s/step - loss: 2.2724 - accuracy: 0.1167 - val_loss: 2.2196 - val_accuracy: 0.1167
Epoch 3/10
8/8 [==============================] - 40s 5s/step - loss: 2.4860 - accuracy: 0.1333 - val_loss: 2.1967 - val_accuracy: 0.1167
Epoch 4/10
8/8 [==============================] - 41s 5s/step - loss: 2.1978 - accuracy: 0.1125 - val_loss: 2.1969 - val_accuracy: 0.1167
Epoch 5/10
3/8 [==========>...................] - ETA: 25s - loss: 2.4153 - accuracy: 0.1458